In [1]:
import os
import nrrd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from skimage.measure import label, regionprops
import shutil
from skimage.measure import label as ski_label
from skimage.measure import regionprops as ski_regionprops
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# 设置文件夹路径(根目录即可，递归处理子文件夹)
input_folder = r"G:\Program\DATABASE\2023\Xiang\ChenShuShu\ChenShuShu_V_LLLN_B.seg.nrrd"
output_label_folder = r"G:\Program\DATABASE_RECT"

In [3]:
data, header = nrrd.read(input_folder)
print(data.ndim)

# 定义要提取的键列表
keys_to_extract = ['Segment0_Color', 'Segment0_ColorAutoGenerated', 'Segment0_Extent', 
                  'Segment0_ID', 'Segment0_LabelValue', 'Segment0_Layer', 
                  'Segment0_Name', 'Segment0_NameAutoGenerated', 'Segment0_Tags']

# 使用循环和 f-string 打印每个键和对应的值
for key in keys_to_extract:
    value = header.get(key)
    print(f"{key}: {value}")


4
Segment0_Color: 0.945098 0.839216 0.568627
Segment0_ColorAutoGenerated: 1
Segment0_Extent: 330 411 226 309 75 96
Segment0_ID: Segment_2
Segment0_LabelValue: 2
Segment0_Layer: 0
Segment0_Name: Segment_2
Segment0_NameAutoGenerated: 1
Segment0_Tags: Segmentation.Status:inprogress|TerminologyEntry:Segmentation category and type - 3D Slicer General Anatomy list~SCT^85756007^Tissue~SCT^85756007^Tissue~^^~Anatomic codes - DICOM master list~^^~^^|


In [4]:
from collections import Counter
if data.ndim == 3:
    x_size, y_size, z_slices = data.shape
    for z_value in range(z_slices):
        z_slice_data = data[:, :, z_value]
        labeled_image, num_labels = ski_label(z_slice_data, return_num=True, connectivity=1)
        regions = ski_regionprops(labeled_image)
        if num_labels != 0:
            print(f"Number of images: {z_value}")
            # 初始化一个累积掩膜
            # accumulated_mask = np.zeros_like(z_slice_data, dtype=data.dtype)
            for region in regions:
                min_row, min_col, max_row, max_col = region.bbox
                region_label_num = region.label
                sub_matrix = z_slice_data[min_row:max_row, min_col:max_col]
                # 统计bbox内的非0像素值
                non_zero_values = sub_matrix[sub_matrix != 0]
                # 统计非0像素值的频率
                value_counts = Counter(non_zero_values)
                print(value_counts)
                # 找到出现次数第一多的像素值及其出现的次数
                label_value, most_common_count = value_counts.most_common(1)[0]
                print(f"Label value: {label_value}")
                sub_matrix[:] = label_value
                z_slice_data[min_row:max_row, min_col:max_col] = sub_matrix
                # print(z_slice_data)
                data[:, :, z_value] = z_slice_data
                
        # if z_value == 37:
        #     # 获取每个连通区域的具体像素值
        #     unique_labels = np.unique(labeled_image)[1:]  # 排除背景标签0
        #     print(f"Number of images: {z_value}")
        #     for label in unique_labels:
        #         # 找到当前标签对应的所有像素位置
        #         pixels = np.argwhere(labeled_image == label)
        #         # 提取这些标签的值
        #         pixel_values = np.unique(z_slice_data[labeled_image == label])


In [21]:
def get_data(three_d_data):
    x_size, y_size, z_slices = three_d_data.shape
    for z_value in range(z_slices):
        z_slice_data = three_d_data[:, :, z_value]
        labeled_image, num_labels = ski_label(z_slice_data, return_num=True, connectivity=1)
        regions = ski_regionprops(labeled_image)
        if num_labels != 0:
            # print(f"Number of images: {z_value}")
            # 初始化一个累积掩膜
            # accumulated_mask = np.zeros_like(z_slice_data, dtype=data.dtype)
            for region in regions:
                min_row, min_col, max_row, max_col = region.bbox
                region_label_num = region.label
                sub_matrix = z_slice_data[min_row:max_row, min_col:max_col]
                # 统计bbox内的非0像素值
                non_zero_values = sub_matrix[sub_matrix != 0]
                # 统计非0像素值的频率
                value_counts = Counter(non_zero_values)
                # print(value_counts)
                # 找到出现次数第一多的像素值及其出现的次数
                label_value, most_common_count = value_counts.most_common(1)[0]
                print(f"Label value: {label_value}")
                sub_matrix[:] = label_value
                z_slice_data[min_row:max_row, min_col:max_col] = sub_matrix
                # print(z_slice_data)
                three_d_data[:, :, z_value] = z_slice_data
    return three_d_data

In [22]:
if data.ndim == 4:
    for i in range(data.shape[0]): # 即为标签数（LN数）
        processed_data = get_data(data[0, :, :, :])
    # x_size, y_size, z_slices = data.shape[1], data.shape[2], data.shape[3]
    # for z_value in range(z_slices):
    #     # 初始化一个累积掩膜，以保留所有 segments 的信息
    #     accumulated_mask = np.zeros((x_size, y_size), dtype=data.dtype)
    #     for segment_idx in range(num_segments):
    #         z_slice_data = data[segment_idx, :, :, z_value]
    #         # 连通区域分析
    #         labeled_image = ski_label(z_slice_data)
    #         regions = ski_regionprops(labeled_image)
    #         for region in regions:
    #             min_row, min_col, max_row, max_col = region.bbox
    #             # 在累积掩膜上标记矩形区域，使用 segment_idx + 1 作为标签值
    #             accumulated_mask[min_row:max_row, min_col:max_col] = segment_idx + 1
    #     # 将累积掩膜赋值回原数据
    #     data[:, :, :, z_value] = 0  # 先清零该切片的所有数据
    #     for segment_idx in range(num_segments):
    #         # 更新每个 segment 的数据
    #         segment_mask = (accumulated_mask == segment_idx + 1).astype(data.dtype)
    #         data[segment_idx, :, :, z_value] = segment_mask
    


Label value: 6
Label value: 6
Label value: 6
Label value: 6
Label value: 6
Label value: 5
Label value: 4
Label value: 4
Label value: 4
Label value: 3
Label value: 3
Label value: 3
Label value: 3
Label value: 3
Label value: 2
Label value: 2
Label value: 7
Label value: 7
Label value: 7
Label value: 1
Label value: 7
Label value: 1
Label value: 1


In [5]:
# root_dir = r"G:\Program\DATABASE"
# four_d_data = []
# for root, dirs, files in os.walk(root_dir):
#     for file in files:
#         if file.endswith(".seg.nrrd"):
#             input_folder = os.path.join(root, file)
#             # 读取数据
#             data, header = nrrd.read(input_folder)
#             if data.ndim == 4:
#                 four_d_data.append(input_folder)
#             

In [18]:
def get_unique_labels(region, z_slice_data):
    min_row, min_col, max_row, max_col = region.bbox
    region_label_num = region.label
    sub_matrix = z_slice_data[min_row:max_row, min_col:max_col]
    # 统计bbox内的非0像素值
    non_zero_values = sub_matrix[sub_matrix != 0]
    # 统计非0像素值的频率
    value_counts = Counter(non_zero_values)
    # print(value_counts)
    # 找到出现次数第一多的像素值及其出现的次数
    label_value, most_common_count = value_counts.most_common(1)[0]
    return label_value


In [20]:
# for path in four_d_data:
#     # print(path)
#     # 读取数据
#     data, header = nrrd.read(path)
#     data1 = data[0, :, :, :]
#     data2 = data[1, :, :, :]
#     x1_size, y1_size, z1_slices = data1.shape
#     x2_size, y2_size, z2_slices = data2.shape
#     for z_value in range(z1_slices):
#         z_slice_data1 = data1[:, :, z_value]
#         z_slice_data2 = data2[:, :, z_value]
#         labeled_image1, num_labels1 = ski_label(z_slice_data1, return_num=True, connectivity=1)
#         labeled_image2, num_labels2 = ski_label(z_slice_data2, return_num=True, connectivity=1)
#         if num_labels1 != 0 and num_labels2 != 0:
#             regions1 = ski_regionprops(labeled_image1)
#             regions2 = ski_regionprops(labeled_image2)
#             for r1, r2 in zip(regions1, regions2):
#                 label_value1 = get_unique_labels(r1, z_slice_data1)
#                 label_value2 = get_unique_labels(r2, z_slice_data2)
#                 print(f"z_value: {z_value}, label_num1:{r1.label}, label_value1:{label_value1}")
#                 print(f"z_value: {z_value}, label_num2:{r2.label}, label_value2:{label_value2}")
#                 print("-"*50)


z_value: 78, label_num1:1, label_value1:6
z_value: 78, label_num2:1, label_value2:2
--------------------------------------------------
z_value: 79, label_num1:1, label_value1:6
z_value: 79, label_num2:1, label_value2:2
--------------------------------------------------
z_value: 80, label_num1:1, label_value1:6
z_value: 80, label_num2:1, label_value2:2
--------------------------------------------------
z_value: 83, label_num1:1, label_value1:5
z_value: 83, label_num2:1, label_value2:2
--------------------------------------------------
z_value: 84, label_num1:1, label_value1:5
z_value: 84, label_num2:1, label_value2:2
--------------------------------------------------
z_value: 85, label_num1:1, label_value1:5
z_value: 85, label_num2:1, label_value2:2
--------------------------------------------------
z_value: 122, label_num1:1, label_value1:2
z_value: 122, label_num2:1, label_value2:1
--------------------------------------------------
z_value: 123, label_num1:1, label_value1:2
z_value: 1

KeyboardInterrupt: 

z_value: 123, label num: 1
z_value: 124, label num: 1
z_value: 125, label num: 1
z_value: 126, label num: 1
z_value: 127, label num: 1
z_value: 128, label num: 1
z_value: 129, label num: 1
z_value: 130, label num: 1
z_value: 131, label num: 1
z_value: 132, label num: 1
z_value: 133, label num: 1
z_value: 134, label num: 1
z_value: 135, label num: 1
z_value: 136, label num: 1
z_value: 137, label num: 1
z_value: 138, label num: 1
z_value: 139, label num: 1
z_value: 140, label num: 1
z_value: 146, label num: 1
z_value: 147, label num: 1
z_value: 148, label num: 1
z_value: 149, label num: 1
z_value: 150, label num: 1
z_value: 151, label num: 1
z_value: 152, label num: 1
z_value: 153, label num: 1
z_value: 154, label num: 1
z_value: 155, label num: 1
z_value: 156, label num: 2
z_value: 157, label num: 2
z_value: 158, label num: 2
z_value: 159, label num: 2
z_value: 160, label num: 2
z_value: 161, label num: 2
z_value: 162, label num: 2
z_value: 163, label num: 2
z_value: 164, label num: 1
z